In [53]:

from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
print("Libraries imported")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Libraries imported


In [57]:
df = pd.read_csv('https://raw.githubusercontent.com/gsamansharma/Telegram-bot-for-spam-detection/main/spam.csv', encoding='ISO-8859-1')
le = LabelEncoder()


In [30]:
data = df[['v1','v2']]

In [31]:

X = data.iloc[:, 1]
y = data.iloc[:, 0]

In [54]:
tokenizer = RegexpTokenizer('\w+')
sw = stopwords.words('english')

In [55]:

ps = PorterStemmer()

In [56]:

def getStem(review):
    review = review.lower()
    tokens = tokenizer.tokenize(review) # breaking into small words
    removed_stopwords = [w for w in tokens if w not in sw]
    stemmed_words = [ps.stem(token) for token in removed_stopwords]
    clean_review = ' '.join(stemmed_words)
    return clean_review

In [35]:
def getDoc(document):
    d = []
    for doc in document:
        d.append(getStem(doc))
    return d

In [36]:

stemmed_doc = getDoc(X)

cv = CountVectorizer()

vc = cv.fit_transform(stemmed_doc)

X = vc.todense()



In [37]:

from sklearn import model_selection

In [11]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:

from sklearn.naive_bayes import MultinomialNB
import pickle

In [13]:

naivee = MultinomialNB()
naivee.fit(X_train, y_train)
naivee.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


0.977705274605764

In [46]:
df['transformed_text'] = df['v2'].apply(getStem)

In [47]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv = CountVectorizer()
tfidf = TfidfVectorizer(max_features=3000)

X = tfidf.fit_transform(df['transformed_text']).toarray()

In [48]:

pickle.dump(naivee, open('model.pkl', 'wb'))
pickle.dump(tfidf,open('vectorizer.pkl','wb'))

In [19]:
model = pickle.load(open('model.pkl', 'rb'))

In [15]:

message = [
    """
    England v Macedonia - dont miss the goals/team news. Txt ur national team to 87077 eg ENGLAND to 87077 Try:WALES, SCOTLAND 4txt/̼1.20
    """
]

In [16]:
def prepare(message):
    d = getDoc(message)
    # dont do fit_transform!! it will create new vocab.
    return cv.transform(d)

In [17]:

message = prepare(message)


In [20]:
y_pred = model.predict(message)
print(y_pred)

['spam']
